In [1]:
!pip install langsmith > /dev/null
!pip install langchain > /dev/null
!pip install openai > /dev/null

Das System kann den angegebenen Pfad nicht finden.
Das System kann den angegebenen Pfad nicht finden.
Das System kann den angegebenen Pfad nicht finden.


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI()
llm.predict("How are you?")

'Hello! How can I assist you today?'

### Using Langsmith

In [2]:
from langchain.callbacks.tracers import LangChainTracer

tracer = LangChainTracer(project_name="My Project")
llm.predict("How many people live in USA?", callbacks=[tracer])

NameError: name 'llm' is not defined

In [ ]:
from langchain.callbacks import tracing_v2_enabled

with tracing_v2_enabled(project_name="My Project"):
    llm.predict("How many people live in USA?")

In [ ]:
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0, tags=["my-llm-tag"])
prompt = PromptTemplate.from_template("Say {input}")
chain = LLMChain(llm=llm, prompt=prompt, tags=["one-tag", "another-tag"])

chain("Hello, World!", tags=["shared-tags"])

{'input': 'Hello, World!', 'text': 'Hello, World!'}

In [ ]:
from langchain.callbacks.manager import (
    trace_as_chain_group
)

with trace_as_chain_group("my_group_name") as group_manager:
    pass

from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0.9)
prompt = PromptTemplate(
    input_variables=["question"],
    template="What is the answer to {question}?",
)
chain = LLMChain(llm=llm, prompt=prompt)
with trace_as_chain_group("my_group") as group_manager:
    chain.run(question="What is your name?", callbacks=group_manager)
    chain.run(question="What is your quest?", callbacks=group_manager)
    chain.run(question="What is your favorite color?", callbacks=group_manager)

In [ ]:
from langsmith import Client

client = Client()
project_runs = client.list_runs(project_name="default")
project_runs

<generator object Client.list_runs at 0x000001DE46F7DE70>

In [ ]:
from datetime import datetime, timedelta

todays_runs = client.list_runs(
    start_time=datetime.now() - timedelta(days=1),
    run_type="llm",
)
todays_runs


<generator object Client.list_runs at 0x000001DE46F7E150>

In [ ]:
for run in todays_runs:
    print(run)

id=UUID('77c6d7e9-4b96-4206-9547-32dd58d0d199') name='ChatOpenAI' start_time=datetime.datetime(2023, 8, 15, 4, 50, 8, 94911) run_type='llm' end_time=datetime.datetime(2023, 8, 15, 4, 50, 8, 727853) extra={'options': {'stop': None}, 'runtime': {'library': 'langchain', 'runtime': 'python', 'platform': 'Windows-10-10.0.19045-SP0', 'sdk_version': '0.0.22', 'library_version': '0.0.264', 'runtime_version': '3.11.0', 'langchain_version': '0.0.264'}, 'total_tokens': 16, 'prompt_tokens': 12, 'completion_tokens': 4, 'invocation_params': {'n': 1, 'stop': None, '_type': 'openai-chat', 'model': 'gpt-3.5-turbo', 'stream': False, 'max_tokens': None, 'model_name': 'gpt-3.5-turbo', 'temperature': 0.0, 'request_timeout': None}} error=None serialized=None events=[{'name': 'start', 'time': '2023-08-15T04:50:08.094911'}, {'name': 'end', 'time': '2023-08-15T04:50:08.727853'}] inputs={'messages': [{'id': ['langchain', 'schema', 'messages', 'HumanMessage'], 'lc': 1, 'type': 'constructor', 'kwargs': {'content'

### Metadata
Add metadata to filter runs later, for example for making an A/B test


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

chat_model = ChatOpenAI()
chain = LLMChain.from_string(llm=chat_model, template="What's the answer to {input}?")

chain({"input": "What is the meaning of life?"}, metadata={"variant": "abc123"})

{'input': 'What is the meaning of life?',
 'text': "The meaning of life is a philosophical and existential question that has been debated by scholars, thinkers, and individuals throughout history. Different people and cultures have offered various interpretations and perspectives on this question. Some believe that the meaning of life is to seek happiness and fulfillment, while others argue it is to find purpose and make a positive impact in the world. Ultimately, the answer to this question may vary depending on one's personal beliefs, values, and experiences."}

In [ ]:
from langsmith import Client

client = Client()
runs = list(client.list_runs(
    filter='has(metadata, \'{"variant": "abc123"}\')',
))
print(list(runs))

[Run(id=UUID('acab4684-cf04-4b5f-ac7d-a071e99b0544'), name='ChatOpenAI', start_time=datetime.datetime(2023, 8, 15, 6, 32, 41, 712821), run_type='llm', end_time=datetime.datetime(2023, 8, 15, 6, 32, 45, 102391), extra={'options': {'stop': None}, 'runtime': {'library': 'langchain', 'runtime': 'python', 'platform': 'Windows-10-10.0.19045-SP0', 'sdk_version': '0.0.22', 'library_version': '0.0.264', 'runtime_version': '3.11.0', 'langchain_version': '0.0.264'}, 'metadata': {'variant': 'abc123'}, 'total_tokens': 109, 'prompt_tokens': 19, 'completion_tokens': 90, 'invocation_params': {'n': 1, 'stop': None, '_type': 'openai-chat', 'model': 'gpt-3.5-turbo', 'stream': False, 'max_tokens': None, 'model_name': 'gpt-3.5-turbo', 'temperature': 0.7, 'request_timeout': None}}, error=None, serialized=None, events=[{'name': 'start', 'time': '2023-08-15T06:32:41.712821'}, {'name': 'end', 'time': '2023-08-15T06:32:45.102391'}], inputs={'messages': [{'id': ['langchain', 'schema', 'messages', 'HumanMessage']